In [0]:
from pyspark.sql.functions import (
    year, month, count, expr, to_date, 
    monotonically_increasing_id, date_add, lit, trunc
)
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [0]:
# --- 1. CARGAR, FILTRAR Y AGREGAR DATOS MENSUALMENTE ---
df = spark.table("workspace.default.complete_dataset_con_clima")

# Filtrar para incluir solo casos de Chikungunya
df_chikungunya = df.filter(df["clasificacion_final"] == "Chikungunya")

# Agrupar por mes en lugar de semana
df_monthly = (
    df_chikungunya
      .withColumn("Mes", trunc("fecha_notificacion", "month"))
      .groupBy("Mes")
      .agg(count("*").alias("y"))
      .orderBy("Mes")
)

pdf = df_monthly.toPandas()

In [0]:
# --- 2. INGENIERÍA DE CARACTERÍSTICAS (FEATURES) ---
# Asegurar que "Mes" sea de tipo datetime
pdf["Mes"] = pd.to_datetime(pdf["Mes"])

# Crear la variable de tendencia temporal
pdf["t"] = np.arange(len(pdf))

# Crear características de estacionalidad basadas en el mes (ciclo de 12)
pdf["month_of_year"] = pdf["Mes"].dt.month.astype(float)
pdf["sin_month"] = np.sin(2 * np.pi * pdf["month_of_year"] / 12)
pdf["cos_month"] = np.cos(2 * np.pi * pdf["month_of_year"] / 12)

# Definir las variables para el modelo
X = pdf[["t", "sin_month", "cos_month"]]
X = sm.add_constant(X)
y = pdf["y"]

In [0]:
# --- 3. ENTRENAR EL MODELO ---
model = sm.OLS(y, X).fit()

In [0]:
# --- 4. PREPARAR DATOS FUTUROS ---

meses_a_predecir = 24 

last_date = pdf["Mes"].max()
last_t = pdf["t"].max()

# Genera el número dinámico de fechas futuras
future_dates = [last_date + pd.DateOffset(months=i+1) for i in range(meses_a_predecir)]
future_pdf = pd.DataFrame(future_dates, columns=["Mes"])

# Crea las características para el número dinámico de meses futuros
future_pdf["t"] = [last_t + i + 1 for i in range(meses_a_predecir)]
future_pdf["month_of_year"] = future_pdf["Mes"].dt.month.astype(float)
future_pdf["sin_month"] = np.sin(2 * np.pi * future_pdf["month_of_year"] / 12)
future_pdf["cos_month"] = np.cos(2 * np.pi * future_pdf["month_of_year"] / 12)

future_X = sm.add_constant(future_pdf[["t", "sin_month", "cos_month"]])

In [0]:
# --- 5. PREDECIR VALORES FUTUROS ---
future_pdf["casos_previstos"] = model.predict(future_X)

In [0]:
# --- 6. VISUALIZACIÓN DE RESULTADOS ---
print("\n--- Visualizando los resultados del modelo ---")
plt.figure(figsize=(18, 8))

# Datos históricos reales
plt.plot(pdf['Mes'], pdf['y'], label='Casos Reales Mensuales', color='blue', linewidth=2)

# Ajuste del modelo sobre los datos históricos
plt.plot(pdf['Mes'], model.fittedvalues, label='Ajuste del Modelo (Histórico)', color='red', linestyle='--')

# Predicciones futuras (la etiqueta ahora es dinámica)
plt.plot(future_pdf['Mes'], future_pdf['casos_previstos'], label=f'Predicción Futura ({meses_a_predecir} meses)', color='green', marker='o', linestyle='--')

# Títulos y etiquetas del gráfico
plt.title('Casos Mensuales de Chikungunya: Reales vs. Predichos', fontsize=16)
plt.xlabel('Mes', fontsize=12)
plt.ylabel('Número de Casos', fontsize=12)
plt.legend(fontsize=10)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# El mensaje de la tabla también es dinámico
print(f"\n--- PREDICCIONES DE CASOS PARA LOS PRÓXIMOS {meses_a_predecir} MESES ---")
display(future_pdf[['Mes', 'casos_previstos']])


In [0]:
# --- GUARDAR LAS PREDICCIONES EN UNA TABLA ---
print("\n--- Guardando las predicciones en una tabla de Spark ---")
df_predicciones_spark = spark.createDataFrame(future_pdf)

nombre_tabla_predicciones = "predicciones_casos_confirmados_chikungunya"
df_predicciones_spark.write.mode("overwrite").saveAsTable(f"workspace.default.{nombre_tabla_predicciones}")

print(f"¡Éxito! Las predicciones se han guardado en la tabla: '{nombre_tabla_predicciones}'")